In [1]:
import pandas as pd

In [36]:
def make_slots(s):
    out_string = ''
    for x in s.split(' '):
        out_string += "O"
        out_string += " "*len(x)

    return out_string

In [13]:
df = pd.read_csv('raw_data_large.csv', sep = ',')
df

,Text,label
0,Are my devices experiencing consistently high ...,time_series
1,Are my devices experiencing consistently high ...,time_series
2,Are there patterns of increased CPU usage duri...,time_series
3,Are there specific times of the day that exper...,time_series
4,Are there any drops in network over the last x...,time_series
5,When were peak drops (errors or discards) over...,time_series
6,What was the bandwidth utilization during the ...,time_series
7,Can you provide the timeline where the network...,time_series
8,Can you provide the timeline where the network...,time_series
9,Which HWSKU is contributing to the maximum net...,time_series


In [10]:
# df.to_csv('raw_data_1.csv', sep = ',', index = False)

### Create Intent Label File

In [20]:
f  = open('intent_label.txt', 'w')
for v in df.label.unique():
    f.write(v + '\n')
f.write('UNK')
f.close()

### Train Labels

In [22]:
f  = open('./train/label', 'w')
for v in df.label.values:
    f.write(v + '\n')

f.close()

### Train seq IN

In [23]:
f  = open('./train/seq.in', 'w')
for v in df.Text.values:
    f.write(v + '\n')

f.close()

### Train seq OUT

In [24]:
f  = open('./train/seq.out', 'w')
for v in df.Text.values:
    slot_len = len(v.split(' '))
    slots_string = ["O"]*slot_len
    slots_string = " ".join(slots_string)
    f.write(slots_string + '\n')

f.close()

#### Manual Labeling DataSet

In [39]:
f  = open('./train/manuallabelset.out', 'w')
for v in df.Text.values:
    f.write(v + '\n')
    f.write(make_slots(v) + '\n')

f.close()

#### Manually Slot Labeled Set

In [40]:
f  = open('./train/manual_labeled_phani.txt', 'r')
file_content = f.read()
f.close()

In [113]:
slot_labeled_data_list = []
for i, s in enumerate(file_content.split('\n')):
    if i >= 1 and i%2 != 0:
        slot_labeled_data_list.append(" ".join(s.split()))

slot_labeled_data_list  

['O O O O O O O O B-entity_type O',
 'O O O O O O O O B-entity_type O',
 'O O O O O B-entity_type O O O O O',
 'O O O O O O O O O O O B-entity_type I-entity_type',
 'O O O O O B-entity_type O O O B-time_frame I-time_frame',
 'O O O O O O O O O O B-time_frame I-time_frame',
 'O O O O O O O B-time_frame I-time_frame',
 'O O O O O O O B-entity_type I-entity_type O O O O O O B-time_frame I-time_frame O O O',
 'O O O O O O O B-entity_type I-entity_type O O O O O O B-time_frame I-time_frame I-time_frame O O O',
 'O O O O O O O B-entity_type I-entity_type O O B-time_frame I-time_frame I-time_frame',
 'O O B-time_frame I-time_frame I-time_frame I-time_frame I-time_frame O O O O B-entity_type I-entity_type',
 'O O B-entity_type O O O O O O O',
 'O B-entity_type O O O O O O O O',
 'O O O O O O O O B-entity_type I-entity_type',
 'O O B-entity_type O O O O O O O',
 'O O O B-entity_type O O O O O',
 'O O B-entity_type O O O',
 'O O B-entity_type O O O O O',
 'O O B-entity_type O O O O O',
 'O O B-c

In [71]:
f  = open('./train/seq.out', 'w')
for v in slot_labeled_data_list:
    f.write(v + '\n')
f.close()

In [110]:
slot_labels = sorted(list(set(sum([v.split() for v in slot_labeled_data_list], []))))
slot_labels = ["-".join(x[::-1]) for x in sorted([x.split('-')[::-1] for x in slot_labels])]
slot_labels

['O',
 'B-count_what',
 'I-count_what',
 'B-entity_type',
 'I-entity_type',
 'B-list_what',
 'B-time_frame',
 'I-time_frame']

In [111]:
# ["-".join(x[::-1]) for x in sorted([x.split('-')[::-1] for x in slot_labels])]

In [112]:
f  = open('slot_label.txt', 'w')
f.write('PAD' + '\n')
f.write('UNK' + '\n')

for v in slot_labels:
    f.write(v + '\n')

f.close()

## Create Dev and Test data

In [80]:
!pwd

/Users/maddurup/Desktop/ResearchNotebooks/pytorch-joint-bert/data/katanemo


In [82]:
!cp -r ./train/s* ./dev/
!cp -r ./train/l* ./dev/

In [83]:
!cp -r ./train/s* ./test/
!cp -r ./train/l* ./test/

In [28]:
s = "Are my devices experiencing consistently"
s

'Are my devices experiencing consistently'

In [30]:
s.split(' ')

['Are', 'my', 'devices', 'experiencing', 'consistently']

In [34]:
def make_slots(s):
    out_string = ''
    for x in s.split(' '):
        out_string += "O"
        out_string += " "*len(x)

    return out_string

In [35]:
make_slots(s)

'O   O  O       O            O            '